In [1]:
import numpy as np

from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.time import Time
from skyfield.api import load

In [2]:
# Set the path
path_figure = '../Plot_figures/'

### Define some functions

In [3]:
# The projections of the parallactic ellipse
def get_project_parallactic(ra_deg, dec_deg, time_jd):
    
    '''
    Purpose:
    Get the projection over ra, dec of the parallactic eclipse (f_ra, f_dec) 
    from the given ra, dec, and the observation time
    
    Input:
    - ra_deg        [float; ndarray]: Right Ascension [deg]
    - dec_deg       [float; ndarray]: Declination [deg]
    - time_jd       [float; ndarray]: Time of the observation [Julian day]
    
    Output: 
    - f_ra          [float; ndarray]: projection over ra of the parallactic eclipse [au]
    - f_dec         [float; ndarray]: projection over dec of the parallactic eclipse [au]
    '''
    
    #--------------------------------------- 
    # Step 1:
    # Get the Barycentric coordinates of the Earth [au] from the given time [Julian day]
    
    # The position of the Earth (python package: Skyfield)
    planets = load('de421.bsp')
    earth   = planets['earth']

    ts = load.timescale()
    t  = ts.ut1_jd(time_jd)

    # BCRS positions of Earth [au]
    # From the center of the Solar System (Barycentric coord.)
    earth_bcrs    = earth.at(t)
    earth_bcrs_au = earth_bcrs.position.au
       
    #--------------------------------------- 
    # Step 2:
    # Get the projection over ra, dec of the parallactic eclipse (f_ra, f_dec)
    
    # Corvert ra, dec from degree to rad
    ra_rad  = np.deg2rad(ra_deg)
    dec_rad = np.deg2rad(dec_deg)
    
    # Corvert BCRS_earth to X, Y, Z format
    X = earth_bcrs_au[0]  # [au]
    Y = earth_bcrs_au[1]  # [au]
    Z = earth_bcrs_au[2]  # [au]
    
    # projection over ra and dec of the parallactic eclipse (Seidelmann 1992)
    f_ra  = np.divide(X*np.sin(ra_rad)-Y*np.cos(ra_rad), np.cos(dec_rad))
    f_dec = X*np.cos(ra_rad)*np.sin(dec_rad) + Y*np.sin(ra_rad)*np.sin(dec_rad) - Z*np.cos(dec_rad)

    return f_ra, f_dec

In [4]:
def cal_radec(ra_ini_deg, dec_ini_deg, ra_0_deg, dec_0_deg, prop_ra, prop_dec, prllx, time_jd, time_diff):
    
    '''
    Purpose:
    Calculate the ra & dec from the original guessing ra_ini & dec_ini
    
    Input:
    - ra_ini_deg        [float]: Initial Guess: Right Ascension [deg]
    - dec_ini_deg       [float]: Initial Guess: Declination [deg]
    - ra_0 _deg         [float]: Fitting result: Right Ascension [deg]
    - dec_0_deg         [float]: Fitting result: Declination [deg]
    - prop_ra           [float]: Proper motion in the direction of ra [mas yr-1]
    - prop_dec          [float]: Proper motion in the direction of dec [mas yr-1]
    - prllx             [float]: Parallax [mas yr-1]
    - time_jd           [float]: Time of the observation [Julian day]
    - time_diff         [float]: Time different: obs. of Fitting & Initial Guess [yr]
    
    
    Return:
    - ra_fin            [float]: Finial value: Right Ascension [deg]
    - dec_fin           [float]: Finial value: Declination [deg]
    
    '''
    
    # transfer the unit
    prllx_deg    = mas2deg(prllx)
    prop_ra_deg  = mas2deg(prop_ra)
    prop_dec_deg = mas2deg(prop_dec)
    
    # get the projection over ra and dec of the parallactic eclipse (Seidelmann 1992)
    f_ra, f_dec = get_project_parallactic(ra_ini_deg, dec_ini_deg, time_jd)
    
    ra_fin_deg  = ra_0_deg  + prop_ra_deg*time_diff  + prllx_deg*f_ra
    dec_fin_deg = dec_0_deg + prop_dec_deg*time_diff + prllx_deg*f_dec
    
    return ra_fin_deg, dec_fin_deg



def get_radec(ra_ini_deg, dec_ini_deg, ra_0_deg, dec_0_deg, \
             prop_ra, prop_dec, prllx, time_ini_yr, time_fin_yr, time_num, epsilon= 1e-20):
    
    '''
    Purpose:
    Get the ra & dec from the original guessing ra_ini & dec_ini, 
    and stop at the |ra_fin - ra_ini| < err & |dec_fin - dec_ini| < err
    
    Input:
    - ra_ini_deg        [float]: Initial Guess: Right Ascension [deg]
    - dec_ini_deg       [float]: Initial Guess: Declination [deg]
    - ra_0_deg          [float]: Fitting result: Right Ascension [deg]
    - dec_0_deg         [float]: Fitting result: Declination [deg]
    - prop_ra           [float]: Proper motion in the direction of ra [mas yr-1]
    - prop_dec          [float]: Proper motion in the direction of dec [mas yr-1]
    - prllx             [float]: Parallax [mas yr-1]
    - time_ini_yr       [float]: Initial time [yr]
    - time_fin_yr       [float]: Final time [yr]
    - time_num            [int]: Number of time interval bwt ini. & fin. time [#]
    - epsilon           [float]: error
       
    
    Return:
    - ra_deg_plot_arr   [float]: Finial value: Right Ascension [deg]
    - dec_deg_plot_arr  [float]: Finial value: Declination [deg]
    '''
    
    # create Time list [yr]
    time_yr_list = np.linspace(time_ini_yr, time_fin_yr, time_num)    
    
    # create empty lists
    ra_deg_plot_list  = []
    dec_deg_plot_list = []
    
    for i, time_yr in enumerate(time_yr_list):
        
        # Time
        time_jd      = Time(time_yr, format='decimalyear').jd   # observation time [Julian day]
        time_diff_yr = time_yr - time_yr_std                    # Time interval [yr]
    
    
        # set the initial different ra & dec as 100
        ra_diff_mas  = 100.
        dec_diff_mas = 100.
        
        # get the ra & dec with parallax
        while ra_diff_mas > epsilon and dec_diff_mas > epsilon:

            # get the ra & dec from the function & input guessing ra & dec
            ra_fin_deg, dec_fin_deg = cal_radec(ra_ini_deg, dec_ini_deg, ra_0_deg, dec_0_deg, \
                                                prop_ra, prop_dec, prllx, time_jd, time_diff_yr)

            # the difference between final & input ra, dec
            ra_diff_mas  = np.abs(deg2mas(ra_fin_deg - ra_ini_deg))
            dec_diff_mas = np.abs(deg2mas(dec_fin_deg - dec_ini_deg))

            # replace the ra & dec using the result from the cal_radec function 
            ra_ini_deg   = ra_fin_deg
            dec_ini_deg  = dec_fin_deg


        # add the finial ra & dec to list
        ra_deg_plot_list.append(ra_fin_deg)
        dec_deg_plot_list.append(dec_fin_deg)
    
        # refresh the initial ra & dec
        ra_ini_deg   = ra_fin_deg
        dec_ini_deg  = dec_fin_deg
    
    # convert the list to np.array
    ra_deg_plot_arr  = np.array(ra_deg_plot_list)
    dec_deg_plot_arr = np.array(dec_deg_plot_list)
    
    
    return ra_deg_plot_arr, dec_deg_plot_arr

In [5]:
def get_parm_from_lstqrt_linear(x, y, wt=None):
    
    '''
    
    Purpose: Get the fitting parameters from the least-squares 
             Linear regression: y = m*x + c
    
    Input:
    - x         [ndarray]: Input x-coordinates of sample points
    - y         [ndarray]: Input y-coordinates of sample points
    - wt        [ndarray]: Weights to apply to the y-coordinates of the sample points
                           Default: ones array
    
    Return:
    - m           [float]: coefficient m (slope)
    - c           [float]: coefficient c (intercept)
    - m_err       [float]: standard error of coefficient m
    - c_err       [float]: standard error of coefficient c
    - r           [float]: Correlation coefficient
    - chi2        [float]: Chi square
    - chi2_dof    [float]: Chi square/ Degree of freedom

    '''
    # y = m*x + c
    coeff, cov = np.polyfit(x, y, 1, w=wt, cov=True) 
    coeff_, residuals, rank, singular_values, rcond = np.polyfit(x, y, 1, w=wt, full=True)
    
    
    # coefficient
    m = coeff[0]
    c = coeff[1]
    
    # standard error of coefficients
    m_err = np.sqrt(np.diag(cov))[0]
    c_err = np.sqrt(np.diag(cov))[1]
    
    # Chi square & Chi square/degree of freedom
    chi2     = residuals[0]
    dof      = len(x) -1
    chi2_dof = chi2/dof    
    
    # prepare for plotting correlation coefficient
    # y = c*x + m
    coeff_2, cov_2 = np.polyfit(y, x, 1, w=wt, cov=True) 
    # standard error of coefficients
    m_2 = coeff_2[0]
       
    # Correlation coefficient
    r = m * m_2
    
    return m, c, m_err, c_err, chi2, chi2_dof, r

In [6]:
# split the ra_hms to ra_hour, ra_minute, ra_second (type = str)
def split_ra_hms(ra_hms):
    
    ra_h = float(ra_hms.split('h')[0])
    ra_m = float(ra_hms.split('h')[1].split('m')[0])
    ra_s = float(ra_hms.split('m')[1].split('s')[0])
    
    return ra_h, ra_m, ra_s

# split the ra_hms to ra_hour, ra_minute, ra_second (type = 1d array)
def split_ra_hms_1darr(ra_hms_1darr):
    
    # create empty lists
    ra_h_list, ra_m_list, ra_s_list = [], [], []
    
    for i, elm in enumerate(ra_hms_1darr):
    
        # split the ra_hms to ra_hour, ra_minute, ra_second
        ra_h, ra_m, ra_s = split_ra_hms(ra_hms_1darr[i])

        # add the splitted ra
        ra_h_list.append(ra_h)
        ra_m_list.append(ra_m)
        ra_s_list.append(ra_s)

    # convert list to np.array
    ra_h_arr = np.array(ra_h_list).astype(np.float)
    ra_m_arr = np.array(ra_m_list).astype(np.float)
    ra_s_arr = np.array(ra_s_list).astype(np.float)
    
    return ra_h_arr, ra_m_arr, ra_s_arr

# split the dec_dms to dec_degree, dec_arc-minute, dec_arc-second (type = str)
def split_dec_dms(dec_dms):
    
    dec_d = float(dec_dms.split('d')[0])
    dec_m = float(dec_dms.split('d')[1].split('m')[0])
    dec_s = float(dec_dms.split('m')[1].split('s')[0])
    
    return dec_d, dec_m, dec_s

# split the dec_dms to dec_degree, dec_arc-minute, dec_arc-second (type = 1d array)
def split_dec_dms_1darr(dec_dms_1darr):
    
    # create empty lists
    dec_d_list, dec_m_list, dec_s_list = [], [], []
    
    for i, elm in enumerate(dec_dms_1darr):
    
        # split the ra_hms to ra_hour, ra_minute, ra_second
        dec_d, dec_m, dec_s = split_dec_dms(dec_dms_1darr[i])

        # add the splitted dec
        dec_d_list.append(dec_d)
        dec_m_list.append(dec_m)
        dec_s_list.append(dec_s)

    # convert list to np.array
    dec_d_arr = np.array(dec_d_list).astype(np.float)
    dec_m_arr = np.array(dec_m_list).astype(np.float)
    dec_s_arr = np.array(dec_s_list).astype(np.float)
    
    return dec_d_arr, dec_m_arr, dec_s_arr

# Convert the unit of ra * dec from (deg, deg) to (split_sec, split_arcsec) (type = float)
def radec_degdeg2secacrs(ra_deg, dec_deg):
    
    # Convert the unit of ra * dec from (deg, deg) to (hms, dms), respectively
    radec_coord  = SkyCoord(ra_deg, dec_deg, frame='icrs', unit='deg')
    ra_hms       = radec_coord.ra.hms
    dec_dms      = radec_coord.dec.dms
    
    ra_sec       = ra_hms[2]
    dec_arcs     = dec_dms[2]
    
    #radec_hmsdms = radec_coord.to_string('hmsdms')
    #ra_hms       = radec_hmsdms.split()[0]
    #dec_dms      = radec_hmsdms.split()[1]

    # split the ra_hms & dec_dms
    #ra_h , ra_m , ra_sec   = split_ra_hms(ra_hms)
    #dec_d, dec_m, dec_arcs = split_dec_dms(dec_dms)
    
    return ra_sec, dec_arcs

# Convert the unit of ra * dec from (deg, deg) to (split_sec, split_arcsec) (type = 1d array)
def radec_degdeg2secacrs_arr(ra_deg_1darr, dec_deg_1darr):
    
    # create empty lists
    ra_sec_list, dec_arcs_list = [], []
    
    for i, elm in enumerate(ra_deg_1darr):
        
        # convert the unit of ra * dec from (deg, deg) to (split_sec, split_arcsec)
        ra_sec, dec_arcs = radec_degdeg2secacrs(ra_deg_1darr[i], dec_deg_1darr[i])
        
        # add the split_sec, split_arcsec
        ra_sec_list.append(ra_sec)
        dec_arcs_list.append(dec_arcs)
        
    # convert list to np.array
    ra_sec_arr   = np.array(ra_sec_list).astype(np.float)
    dec_arcs_arr = np.array(dec_arcs_list).astype(np.float)
        
    return ra_sec_arr, dec_arcs_arr

In [7]:
# Convert the unit from milli-arcsecond to radius
def mas2rad(u_mas):
    return np.deg2rad(np.divide(u_mas, 36e5))

# Convert the unit from milli-arcsecond to deg
def mas2deg(u_mas):
    return np.divide(u_mas, 36e5)

# Convert the unit from deg to milli-arcsecond
def deg2mas(u_deg):
    return u_deg*36e5

# Convert the ra unit from second to arcsec
def ra_sec2arcs(u_sec):
    return u_sec*15.

# Convert the ra unit from arcsec to second
def ra_arcs2sec(u_arcs):
    return u_arcs/15.

# Split the array for the fitting
def split_Qband(input_arr):
    # fitting without Q band observation due to the poor resolution
    output_arr = input_arr[1:]
    return output_arr

# Find the nearst value in np.array
def find_nearest(inp_arr, inp_value):
    idx = np.abs(inp_arr - inp_value).argmin()
    out_value = inp_arr.flat[idx]
    return idx, out_value

### load the txtfile

In [8]:
# Load the Position txt file
Postxt_name        = 'Position_4A2.txt'
Pos_txt_array      = np.loadtxt(Postxt_name, dtype = str)

# Obtain the values from the Position txt file
Band_arr           = Pos_txt_array.T[2].astype('str')      # Observed bands
time_yms_arr       = Pos_txt_array.T[4].astype('str')      # Observationl Time [y-m-d]
time_yr_arr        = Pos_txt_array.T[6].astype(np.float)   # Observationl Time [year]
time_jd_arr        = Pos_txt_array.T[8].astype(np.float)   # Observationl Time [Julian Day]

ra_hms_arr         = Pos_txt_array.T[10].astype('str')      # Right Ascension [hms]
dec_dms_arr        = Pos_txt_array.T[12].astype('str')     # Declination     [dms]
ra_dig_deg_arr     = Pos_txt_array.T[14].astype(np.float)  # RA w/ significant digit  [degree]
dec_dig_deg_arr    = Pos_txt_array.T[16].astype(np.float)  # Dec w/ significant digit [degree]
ra_deg_arr         = Pos_txt_array.T[18].astype(np.float)  # Right Ascension [degree]
dec_deg_arr        = Pos_txt_array.T[20].astype(np.float)  # Declination     [degree]
ra_mas_arr         = deg2mas(ra_deg_arr)                   # Right Ascension [mas]
dec_mas_arr        = deg2mas(dec_deg_arr)                  # Declination     [mas]
ra_h_arr ,  ra_m_arr,  ra_s_arr = split_ra_hms_1darr(ra_hms_arr)
dec_d_arr, dec_m_arr, dec_s_arr = split_dec_dms_1darr(dec_dms_arr)

radec_err_deg_arr  = Pos_txt_array.T[22].astype(np.float)  # Uncertainties of RA & Dec [degree]
radec_err_arcs_arr = radec_err_deg_arr * 3600              # Uncertainties of RA & Dec [arcsec]
radec_err_mas_arr  = deg2mas(radec_err_deg_arr)            # Uncertainties of RA & Dec [mas]

print (Band_arr) 

['Q' 'K' 'K' 'Ka' 'B7' 'B6' 'B3' 'B4']


### Set some parameters

In [9]:
# Color list for plotting the figure
color_list = ['red', 'orange', 'lime', 'green', 'deepskyblue', 'blue', 'purple', 'magenta']

# The parallax [mas] by the distance of 293 pc for the Perseus Molecular Cloud
prllx      = 3.

#--------------------------------------- 
# Set the stander point for comparison
# Standard: ALMA Band 4
i_std = -1

# Time
time_yms_std = time_yms_arr[i_std]
time_jd_std  = time_jd_arr[i_std]
time_yr_std  = time_yr_arr[i_std]
time_std     = Time(time_yms_std)

# RA & Dec
ra_hms_std   = ra_hms_arr[i_std]
ra_deg_std   = ra_deg_arr[i_std]
ra_mas_std   = deg2mas(ra_deg_std)
ra_h_std, ra_m_std, ra_s_std = split_ra_hms(ra_hms_std)
ra_hour_int   = int(ra_h_std)
ra_minute_int = int(ra_m_std)

dec_dms_std  = dec_dms_arr[i_std]
dec_deg_std  = dec_deg_arr[i_std]
dec_mas_std  = deg2mas(dec_deg_std)
dec_d_std, dec_m_std, dec_s_std = split_dec_dms(dec_dms_std)
dec_deg_int   = int(dec_d_std)
dec_arcm_int  = int(dec_m_std)

Coord_std    = SkyCoord(ra_hms_std, dec_dms_std, frame='icrs')